# Investments results inspection
Routines for inspecting and visualizing _pommesinvest_ investment results.
* Plot investment decisions
* Plot decomissionings
* Plot total installed capacity
* Include exogenous capacities to evaluate plants status

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, aggregate_investment_decision_results, plot_single_investment_variable
)

## Read in data
* Define model configuration / results to be analyzed
* Define column order and colors for evaluation
* Read in the actual results

In [ ]:
time_frame_in_years = 6
frequency = "24H"
path_results = "./model_results/"
path_processed_data = "./data_out/"
path_plots = "./plots/"

FUELS_EXISTING = {
    "uranium": "#e50000",
    "lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "mixedfuels": "#a57e52",
    "otherfossil": "#d8dcd6",
}

FUELS = {
    "biomass": "#15b01a",
    "hydrogen": "#6fa8dc",
    "natgas": "#ffd966",
    "oil": "#aaa662",
    "waste": "#c04e01"
}

STORAGES_INFLOW = {
    "PHS_inflow": "#0c2aac",
    "battery_inflow": "#f7e09a"
}
STORAGES_OTHER = {
    "PHS_outflow": "#0c2aac",
    "battery_outflow": "#f7e09a",
    "PHS_capacity": "#0c2aac",
    "battery_capacity": "#f7e09a"
}

DEMAND_RESPONSE_CLUSTERS = {
    "hoho_cluster_shift_only": "#333333", 
    "hoho_cluster_shift_shed": "#555555", 
    "ind_cluster_shed_only": "#666666",
    "ind_cluster_shift_only": "#888888", 
    "ind_cluster_shift_shed": "#aaaaaa", 
    "tcs+hoho_cluster_shift_only": "#cccccc",
    "tcs_cluster_shift_only": "#dddddd", 
}

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}_investment.csv"
)

filename_backup = "installed_capacity_backup_generation.csv"
filename_storage = "installed_storage_capacity.csv"

plt.rcParams.update({'font.size': 12})

investment_results_raw = pd.read_csv(f"{path_results}{filename}", index_col=0)

# Control workflow
include_storages_in_plots = True

## Preprocess and aggregate data
See imported functions for documentation

In [ ]:
processed_results = preprocess_raw_results(investment_results_raw)
aggregated_results, other_storage_results = aggregate_investment_decision_results(
    processed_results, energy_carriers=FUELS, by="energy_carrier"
)

## Visualize results
Results visualization is separated between investments in storage energy content as well as storage outflow power and the remainder, i.e. investments in generation units, investments in storage inflow power and investments in demand response clusters.

The following parameters are displayed
* Newly installed capacity
* Decommissioned capacity
* Total installed capacity

### Generation, storage inflow and demand response

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable(
    aggregated_results,
    "invest",
    colors=colors,
    aggregation="energy_carrier"
)

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable(
    aggregated_results,
    "old",
    colors=colors,
    aggregation="energy_carrier"
)

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable(
    aggregated_results,
    "total",
    colors=colors,
    aggregation="energy_carrier"
)

### Storage outflow and storage energy content

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "invest",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True
)

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "old",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True
)

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "total",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True
)

## Include exogenous capacities in assessment
* Read in capacities of backup plants, aggregated by fuel
* Read in installed power and capacity of storage units

In [ ]:
backup_generation = pd.read_csv(
    f"{path_processed_data}{filename_backup}", index_col=0
)
backup_generation.index = backup_generation.index.astype(str)

In [ ]:
grouped_total = aggregated_results[["total"]].reset_index()
grouped_total = grouped_total.pivot(
    index="energy_carrier", columns="year", values="total"
).T

In [ ]:
overall_capacity = grouped_total.add(backup_generation)
overall_capacity.loc[:,overall_capacity.isna().all()] = backup_generation
overall_capacity.loc[:,overall_capacity.isna().all()] = grouped_total

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS_EXISTING, **FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS_EXISTING, **FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable(
    overall_capacity.T,
    "all",
    colors=colors,
    group=False
)